In [80]:
import requests
import json
import pandas as pd
import time
from IPython.display import clear_output, display
import sys
import random
import numpy as np
import os
import sqlite3
# Strategy
# get list of players
# from list of players, get list of games
# from list of games, get game stats
from utils import apikey, payload, rate, regions

def write_list(l, filename):
	filename = add_txt_ext(filename)
	if os.path.dirname(filename)!='':
		os.makedirs(os.path.dirname(filename), exist_ok=True)
	with open(filename,'w',encoding='utf-8',errors='ignore') as f:
		for item in l:
			f.write(str(item)+'\n')

def load_list(filename):
	out = []
	with open(filename,'r', encoding='utf-8', errors ='ignore') as f:
		l = f.readline()
		while l!="":
			l = l.strip()
			out.append(l)
			l = f.readline()
	return out
def append_to_file(val, filename):
	filename = add_txt_ext(filename)
	if os.path.dirname(filename)!='':
		os.makedirs(os.path.dirname(filename), exist_ok=True)
	with open(filename, 'a',encoding='utf-8',errors='ignore') as f:
		if type(val)==type(list()):
			for v in val:
				f.write(str(v)+"\n")
		else:
			f.write(str(val)+"\n")
def add_txt_ext(filename):
	path,ext = os.path.splitext(filename)
	if ext != "txt":
		filename = path+".txt"
	return filename

In [142]:
region = 'na1'
rank = 'PLATINUM'
division = 'I'
page = 1
req = f'https://{region}.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{rank}/{division}?page={page}'
resp = requests.get(req,headers=payload)
data = json.loads(resp.content)


[{'leagueId': 'e9535aff-8d6f-4d83-aa68-be700453edf8',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'PLATINUM',
  'rank': 'I',
  'summonerId': 'NFBhoMN_-HiwlJUc3OXWC296oimfQALs0hHaBtaAnnplalol',
  'summonerName': 'xTc4',
  'leaguePoints': 75,
  'wins': 99,
  'losses': 95,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False},
 {'leagueId': 'e068725c-8658-497c-97b7-102f423737e0',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'PLATINUM',
  'rank': 'I',
  'summonerId': 'PpVDo1HuQsxs7719ikjocj1d6ohItOppp5C6JxhPhT_I0Kui',
  'summonerName': 'cleborp100',
  'leaguePoints': 75,
  'wins': 510,
  'losses': 512,
  'veteran': True,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': True},
 {'leagueId': 'ced18464-d729-4402-8f39-7f7e43c53865',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'PLATINUM',
  'rank': 'I',
  'summonerId': 'rb80K8G_Zp3ruUqKwnsl5TSpulFJqXlRGSiMATEizssPt3gd',
  'summonerName': 'Violence Li',
  'leaguePoints': 75,
  'wins': 148,
  'losses': 112

In [148]:
filename = "./League of Legends/data/text.csv"
with open(filename,'a+') as f:
    for d in data:
        f.write(",".join((d['tier'],d['rank'],d['summonerId'])))
        f.write("\n")

In [210]:
# get list of summoners in regular ranks

ranks = ['PLATINUM','DIAMOND']#['IRON','BRONZE','SILVER','GOLD','PLATINUM','DIAMOND']
divisions = ['I','II','III','IV']
for rank in ranks:
    out = pd.DataFrame()
    print(rank)
    for division in divisions:    
        print('\t',division)
        page = 1 #starts at page 1 
        print("\rpage\t",page, end="")        
        resp = requests.get(f'https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{rank}/{division}?page={page}',headers=payload)
        data = json.loads(resp.content)
        time.sleep(1/rate)        
        while resp.ok and data!=[]:            
            df = pd.DataFrame.from_dict(data)
            df = df[['leagueId','tier','rank','summonerId','inactive']]
            out = pd.concat([out,df])
            page+=1
            print("\rpage\t",page, end="")      
            resp = requests.get(f'https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{rank}/{division}?page={page}',headers=payload)    
            data = json.loads(resp.content)
            time.sleep(1/rate)
        print()
    out.to_csv(f"./League of Legends/data/{rank}.csv",index=False)
    
print('done!')
        

PLATINUM
	 I
page	 2	 II
page	 100	 III
page	 155	 IV
page	 435DIAMOND
	 I
page	 15	 II
page	 27	 III
page	 42	 IV
page	 104done!


In [174]:
# get list of summoners in higher ranks
queues = ['RANKED_SOLO_5x5']
queue = 'RANKED_SOLO_5x5'
baseurl = 'https://na1.api.riotgames.com'
leagues = [f'/lol/league/v4/challengerleagues/by-queue/{queue}',
           f'/lol/league/v4/grandmasterleagues/by-queue/{queue}',
           f'/lol/league/v4/masterleagues/by-queue/{queue}'
          ]
names = ['CHALLENGER', 'GRANDMASTER', 'MASTER']
l = 0
for idx, l in enumerate(leagues):
    url = baseurl+l    
    resp = requests.get(url, headers=payload)    
    time.sleep(1/rate)
    data = json.loads(resp.content)
    #display(data)
    df = pd.DataFrame.from_dict(data)
    df['rank']=[a['rank'] for a in df['entries']]
    df['summonerId']=[a['summonerId'] for a in df['entries']]
    df['inactive'] = [a['inactive'] for a in df['entries']]        
    df = df[['leagueId','tier','rank','summonerId','inactive']]
    df['accountId'] = None
    df.to_csv(f'./League of Legends/data/{names[idx]}.csv')
    print(len(data['entries']))



300
700
270


In [175]:
data

{'tier': 'MASTER',
 'leagueId': '136579ee-a212-34b0-b4b8-69ba529f4d7f',
 'queue': 'RANKED_SOLO_5x5',
 'name': "Varus's Villains",
 'entries': [{'summonerId': 'waCItDtn_DA8yEXJdp3DtuIqGZz0abosHinfgaB3mrCgSsg',
   'summonerName': 'Stormwind',
   'leaguePoints': 0,
   'rank': 'I',
   'wins': 289,
   'losses': 285,
   'veteran': False,
   'inactive': False,
   'freshBlood': True,
   'hotStreak': False},
  {'summonerId': 'ybKEnIYa3msb2K62-51Vz900to_lN4rz1rfkSh8Yx-4avX8',
   'summonerName': 'TheBroFist',
   'leaguePoints': 0,
   'rank': 'I',
   'wins': 95,
   'losses': 82,
   'veteran': False,
   'inactive': False,
   'freshBlood': True,
   'hotStreak': False},
  {'summonerId': 'B1VkxVK9aXkCv3QkDATPY9bepvltBWzVB3_UZosDzOz43UhE',
   'summonerName': 'Bxxxx9x Txxx Gxx',
   'leaguePoints': 0,
   'rank': 'I',
   'wins': 107,
   'losses': 52,
   'veteran': False,
   'inactive': False,
   'freshBlood': False,
   'hotStreak': False},
  {'summonerId': '6hhJUyBsDWY3KuXmrzv5xX4CD9V8uGlSVnL70h33X6xZnBc'

In [184]:
# get encrypted account ID from summonerId
summonerID = 'ZrkrmU_B22CJ8UPpZObUj0gI1NF_GlMDgjyri4JKsK2-EsQ'
resp = requests.get(f'https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summonerID}', headers=payload)
data = json.loads(resp.content)
print(data)
accountId = data['accountId']
print(accountId)

{'id': 'ZrkrmU_B22CJ8UPpZObUj0gI1NF_GlMDgjyri4JKsK2-EsQ', 'accountId': 'rQHdKfUYc9lrkkKjYs0iEQWr-tv8PMjI6uxgQBvs52UejQ', 'puuid': 'K1yJ3Hdf-sZoaXKTe-w4zcMzEwTxX9f5OGp-ZyypBCHrKlQUJK4KDXPQ6pnAB0-9FU_hFLt9ngWj-g', 'name': 'RedamancyA', 'profileIconId': 7, 'revisionDate': 1595966761000, 'summonerLevel': 214}
rQHdKfUYc9lrkkKjYs0iEQWr-tv8PMjI6uxgQBvs52UejQ


In [7]:
checkpoint = 1350
# for upper ranks, get account id for summoners
files = ['PLATINUM','DIAMOND','MASTER','GRANDMASTER','CHALLENGER']
for file in files:
    filename = f'./League of Legends/data/{file}.csv'
    df = pd.read_csv(filename)    
    totalItems = len(df)
    print(file)
    count = 0
    for index, row in df.iterrows():        
        print(f'\r\t{index} of {totalItems}',end="")        
        #display(row)
        
        summonerId = row['summonerId']        
        if pd.isnull(row['accountId']):
            count += 1
            resp = requests.get(f'https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summonerId}', headers=payload)            
            time.sleep(1/rate)
            if resp.ok:
                data = json.loads(resp.content)            
                df['accountId'][index] = data['accountId']
            else:
                display(resp)
                raise Exception("bad response from server: "+resp.status_code)
        
        if count>=checkpoint:
            print(f'\ncheckpoint')
            df.to_csv(filename,index=False)
            count = 0
            print()
        
    df.to_csv(filename,index=False)
    print()
    
        
        
    

PLATINUM
	98509 of 140856	140855 of 140856
DIAMOND
	37419 of 37420
MASTER
	284 of 285
GRANDMASTER
	696 of 697
CHALLENGER
	299 of 300


In [73]:
#display(df[['summonerId','accountId']][0:1])
accountId = df['accountId'][0]
req = f'https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/{accountId}'
#print(req)
resp = requests.get(req, headers=payload)
data = json.loads(resp.content)

#display(data)
#display([a['gameId'] for a in data['matches'] if a['queue']==420])
r = list(range(10))
random.shuffle(r)
print(r)

[0, 4, 6, 5, 2, 9, 8, 3, 1, 7]


In [172]:
# for upper ranks, get list of games for summoners (only a sample of plat/diamond)
files = ['PLATINUM','DIAMOND','MASTER','GRANDMASTER','CHALLENGER'] #'PLATINUM',
max_num_of_summoners = 1000
for file in files:
    filename = f'./League of Legends/data/{file}.csv'
    df = pd.read_csv(filename)
    totalItems = len(df)
    print(file)
    count = 0
    games = list()
    rows = list(range(len(df)))
    if len(rows)>max_num_of_summoners:
        random.shuffle(rows)
        rows = rows[0:max_num_of_summoners]
    index = 0
    while index<len(rows):
        row = rows[index]
        print(f"\r{index+1} of {len(rows)}", end="")
        accountId = df['accountId'][row:row+1].values[0]        
        resp = requests.get(f'https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/{accountId}', headers=payload)
        time.sleep(1/rate)
        if resp.ok:
            data = json.loads(resp.content)
            gameIds = [a['gameId'] for a in data['matches'] if a['queue']==420]# or a['queue']==440]
            for g in gameIds:
                games.append(g)
            index += 1
        else:
            display(resp)
            if resp.status_code>=400 and resp.status_code < 420:
                raise Exception(f"bad response from server: {resp.status_code}")
            
    print()
    games = list(set(games))
    out = pd.DataFrame(games)
    out.columns = ['gameId']
    out.to_csv(f'./League of Legends/data/{file}_games.csv', index=False)
        

PLATINUM
494 of 1000

<Response [503]>

521 of 1000

<Response [504]>

947 of 1000

...

149 of 300

<Response [504]>

277 of 300

<Response [504]>

300 of 300


In [139]:
display(data.keys)

{'gameId': 3522963095,
 'platformId': 'NA1',
 'gameCreation': 1596680011573,
 'gameDuration': 1062,
 'queueId': 440,
 'mapId': 11,
 'seasonId': 13,
 'gameVersion': '10.16.330.9186',
 'gameMode': 'CLASSIC',
 'gameType': 'MATCHED_GAME',
 'participants': [{'participantId': 1,
   'teamId': 100,
   'championId': 101,
   'spell1Id': 4,
   'spell2Id': 12,
   'stats': {'participantId': 1,
    'win': False,
    'item0': 1001,
    'item1': 3285,
    'item2': 2423,
    'item3': 1056,
    'item4': 0,
    'item5': 0,
    'item6': 3340,
    'kills': 0,
    'deaths': 3,
    'assists': 3,
    'largestKillingSpree': 0,
    'largestMultiKill': 0,
    'killingSprees': 0,
    'longestTimeSpentLiving': 687,
    'doubleKills': 0,
    'tripleKills': 0,
    'quadraKills': 0,
    'pentaKills': 0,
    'unrealKills': 0,
    'totalDamageDealt': 43595,
    'magicDamageDealt': 37839,
    'physicalDamageDealt': 5756,
    'trueDamageDealt': 0,
    'largestCriticalStrike': 0,
    'totalDamageDealtToChampions': 5413,
 

In [271]:
gameId = '2476737486'
resp = requests.get(f'https://na1.api.riotgames.com/lol/match/v4/matches/{gameId}', headers = payload)
data = json.loads(resp.content)
display(data.keys())
display(data['participantIdentities'][0]['player']['summonerId'])

dict_keys(['status'])

KeyError: 'participantIdentities'

In [32]:
# get game information
blueTeamId = 100
redTeamId = 200
gameId = '3490775054'
def getGameInfo(region, gameId, payload):
    resp = requests.get(f'https://na1.api.riotgames.com/lol/match/v4/matches/{gameId}', headers = payload)
    data = json.loads(resp.content)
    if not resp.ok:
        display(resp)        
        return None, resp.status_code
    #display(data)    
    blue = 0
    red = 1
    out = dict()
    if data['teams'][0]['teamId']==redTeamId: # blue team is 100, red team is 200
        blue = 1
        red = 0
    keys = ['gameId', 'platformId', 'gameCreation', 'gameDuration', 'queueId', 'mapId', 'seasonId', 'gameVersion', 'gameMode',
            'gameType']
    for key in keys:
        out[key] = data[key]
    keys = ['win', 'firstBlood', 'firstTower', 'firstInhibitor','firstBaron','firstDragon','firstRiftHerald']
    for key in keys:
        out[key] = 0
        if data['teams'][blue][key] != False and data['teams'][blue][key] != 'Fail':
            out[key] = 100
        elif data['teams'][red][key] != False and data['teams'][red][key] != 'Fail':
            out[key] = 200
    out['blue_towerKills'] = data['teams'][blue]['towerKills']
    out['red_towerKills'] = data['teams'][red]['towerKills']
    out['blue_inhibitorKills'] = data['teams'][blue]['inhibitorKills']
    out['red_inhibitorKills'] = data['teams'][red]['inhibitorKills']
    out['blue_baronKills'] = data['teams'][blue]['baronKills']
    out['red_baronKills'] = data['teams'][red]['baronKills']
    out['blue_dragonKills'] = data['teams'][blue]['dragonKills']
    out['red_dragonKills'] = data['teams'][red]['dragonKills']
    out['blue_riftHeraldKills'] = data['teams'][blue]['riftHeraldKills']
    out['red_riftHeraldKills'] = data['teams'][red]['riftHeraldKills']    
    #participants = data['participants']
    #blueTeam = [a for a in participants if a['teamId']==blueTeamId]
    #redTeam = [a for a in participants if a['teamId']==redTeamId]
    for idx, participant in enumerate(data['participants']):
        p = reduce_participant_fields(participant)
        if 'player' in data['participantIdentities'][idx].keys() and 'summonerId' in data['participantIdentities'][idx]['player'].keys():
            out[f"p{participant['participantId']}_summonerId"] = data['participantIdentities'][idx]['player']['summonerId']
        else:
            out[f"p{participant['participantId']}_summonerId"] = '0'        
        for key in p.keys():            
            out[f"p{participant['participantId']}_{key}"] = p[key]
        
    return out, resp.status_code


In [9]:
def reduce_participant_fields(participant):

    keys = ['championId','spell1Id','spell2Id']
    out = dict()
    for key in keys:
        if key in participant.keys():
            out[key] = participant[key]
        else:
            out[key] = np.nan

    # from stats:
    keys = ['item0','item1','item2','item3','item4','item5','kills','deaths','assists',
    'totalDamageDealtToChampions','magicDamageDealtToChampions', 'physicalDamageDealtToChampions', 'trueDamageDealtToChampions',
    'totalHeal','damageSelfMitigated','damageDealtToObjectives','damageDealtToTurrets',
    'visionScore','timeCCingOthers','totalDamageTaken','magicalDamageTaken','physicalDamageTaken','trueDamageTaken',
    'goldEarned','goldSpent','turretKills','inhibitorKills','totalMinionsKilled',
    'champLevel','visionWardsBoughtInGame','sightWardsBoughtInGame','wardsPlaced','wardsKilled',
    'perk0','perk1','perk2','perk3','perk4','perk5','perkPrimaryStyle','perkSubStyle']
    for key in keys:
        if key in participant['stats'].keys():
            out[key] = participant['stats'][key]
        else:
            out[key] = np.nan
    # from timeline
    keys=['role','lane']
    for key in keys:
        if key in participant['timeline'].keys():
            out[key] = participant['timeline'][key]    
        else:
            out[key] = np.nan
    
    return out

In [230]:
#df = pd.read_csv('./League of Legends/data/games_na1.csv')
for p in range(10):
    df[f"p{p+1}_summonerId"] = np.nan
'''
columns = list()
existing = df.columns
for e in existing:
    columns.append(e)
for key in game.keys():
    columns.append(key)
df = df.reindex(columns=columns)
'''
display(df)

,tier,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,...,p1_summonerId,p2_summonerId,p3_summonerId,p4_summonerId,p5_summonerId,p6_summonerId,p7_summonerId,p8_summonerId,p9_summonerId,p10_summonerId
0,CHALLENGER,3426025473,NA1,1.589823e+12,1873.0,420.0,11.0,13.0,10.10.320.3039,CLASSIC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CHALLENGER,3515547651,NA1,1.596171e+12,195.0,420.0,11.0,13.0,10.15.330.344,CLASSIC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CHALLENGER,3518726149,NA1,1.596390e+12,1873.0,420.0,11.0,13.0,10.15.330.344,CLASSIC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CHALLENGER,3516497926,NA1,1.596234e+12,1558.0,420.0,11.0,13.0,10.15.330.344,CLASSIC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CHALLENGER,3425927177,NA1,1.589811e+12,1548.0,420.0,11.0,13.0,10.10.320.3039,CLASSIC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144124,PLATINUM,3479044076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144125,PLATINUM,3509190635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144126,PLATINUM,3482845171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144127,PLATINUM,3385327605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [231]:
df.to_csv('./League of Legends/data/games_na1.csv', index = False)

In [35]:
filename = './League of Legends/data/games_na1.csv'
df = pd.read_csv(filename)
index = 0
count = 0 
checkpoint = 1350
max_retries = 10
bad_gameIds = load_list("./League of Legends/bad_gameIds.txt")
retry_count = 0
while index < len(df):            
    gameId = df['gameId'][index]    
    if pd.isnull(df['p1_summonerId'][index]) and not (str(gameId) in bad_gameIds):
        print(f"\r{index} of {len(df)}", end="")
        #print("for",gameId,":", not (str(gameId) in bad_gameIds) and pd.isnull(df['p1_summonerId'][index]))
        row, status = getGameInfo('na1', gameId, payload)
        time.sleep(1/rate)
        if status != 200:
            if status == 403:
                raise Exception('Forbidden 403: Need to re-generate api key')            
            elif status==404:
                
                retry_count+=1
                if retry_count >= max_retries:
                    print("")
                    print(gameId)
                    print('404 not found: exceeded max retries for', df['gameId'][index])
                    #df = df.drop(df.index[index])
                    append_to_file(gameId,'./League of Legends/bad_gameIds.txt')                    
                    index+=1
                    retry_count=0                    
                    
        else:
            retry_count = 0
            for key in row.keys():
                df[key][index] = row[key]
            count += 1
            index += 1
    else:
        index += 1
        retry_count = 0
    
    if count>=checkpoint:
        print(f'\ncheckpoint')
        df.to_csv(filename,index=False)
        count = 0
        print()
    

105386 of 144128

C:\Users\zackw.000\.conda\envs\insight\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


105868 of 144128

<Response [404]>

105868 of 144128

...

144118 of 144128

<Response [404]>

144118 of 144128

<Response [404]>


684982242
404 not found: exceeded max retries for 684982242
144127 of 144128

In [13]:
#filename = './League of Legends/data/games_na1.csv'
#df = pd.read_csv(filename)
bad_gameIds = load_list("./League of Legends/bad_gameIds.txt")
index = 63197
gameId = df['gameId'][index]
row, status = getGameInfo('na1', gameId, payload)
display(row)
#for i in bad_gameIds:
#    print(i)

    


KeyError: 'summonerId'

KeyError: 'summonerId'

In [19]:
data['participantIdentities'][0]['player'].keys()

dict_keys(['platformId', 'accountId', 'summonerName', 'currentPlatformId', 'currentAccountId', 'matchHistoryUri', 'profileIcon'])

In [3]:
file = './data/games_na1.csv'
df = pd.read_csv(file)


C:\Users\zackw.000\.conda\envs\insight\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2,8,9,10,72,73,118,119,164,165,210,211,256,257,302,303,348,349,394,395,440,441,486,487,488,489,490,491,492,493,494,495,496,497) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
len(df)

144128

In [94]:
status = df['gameId'].duplicated()
dupes = status[status==True]
display(dupes)

Series([], Name: gameId, dtype: bool)

In [85]:
display(df.loc[dupes.index[:]])


,tier,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,...,p1_summonerId,p2_summonerId,p3_summonerId,p4_summonerId,p5_summonerId,p6_summonerId,p7_summonerId,p8_summonerId,p9_summonerId,p10_summonerId
143575,PLATINUM,3494509136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143653,PLATINUM,3524524822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143849,PLATINUM,3469737250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143854,PLATINUM,3413638448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144010,PLATINUM,3471965841,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
conn = sqlite3.connect('data/data2.db')
cursor = conn.cursor()
df.to_sql('games',conn,index=False)

In [50]:
gameId = df.loc[[8170]]['gameId']

,tier,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,...,p1_summonerId,p2_summonerId,p3_summonerId,p4_summonerId,p5_summonerId,p6_summonerId,p7_summonerId,p8_summonerId,p9_summonerId,p10_summonerId
8170,GRANDMASTER,3463315468,NA1,1.592286e+12,1499.0,420.0,11.0,13.0,10.12.324.5925,CLASSIC,...,xFusPxMEX5NsWHi1ZDuzsSZu12C7hESa871V961c2DqQqcnU,Ols34X0v5nbM0Vgo4E-Uods_Ch6oh9S-vjk8CChIpRb1,RirpEjhr3Ex5gsa9EwpCxhkrgjdKp7QnGG6caSbtZHNl3Ac,fLtRb8YqWwjWOuDZBqEP6zlUkBernVUCC9k0SGXyOdOsErkb,nwlqG6WMnvSnjVR1pfhM4Chs1pxHG4QdPctAKpFRhDL-QFwP,RAIBxYb5EiNFr1unOvAmoNw-Zfci9qbXtOUtToLhA8TDknKT,Igii9mTGDz4vLEPBH-U1f6cR0ZY1jviJpYWZdapb6LChRDE,Qka5zozhnXskkjK7eV2NcGtGpMeeRf1IiP0nCwd6qN7H8Ao,kmvKw6QowmUHAO4IoTEg8CkhzUOBJ0DamvWYsdYEaq_RFIY,uY6leWqOzokrH5gdvhrKn7MMZWPNG7hYYAgTwftt--Aq8iQ


In [83]:
status = df['gameId'].duplicated()
dupes = status[status==True]
for idx, d_idx in enumerate(dupes.index):
    print(f"\r{idx} of {len(dupes.index)}", end = "")
    gameId = df.loc[d_idx]['gameId']
    t = df[df['gameId']==gameId]
    if len(t)<=1:
        continue
    idx0 = t.index[0]
    idx1 = t.index[1]

    fields = ['gameId','platformId','gameCreation']
    same = True
    for field in fields:
        if t.loc[idx0][field]!=t.loc[idx1][field]:
            same = False
            break

    if same:
        df = df.drop([idx0])


    

4 of 5

In [81]:
df.duplicated()

4         False
6         False
8         False
12        False
14        False
          ...  
144123    False
144124    False
144125    False
144126    False
144127    False
Length: 129109, dtype: bool

In [47]:
dfDupe = dfDupe.append(d)
display(dfDupe)

,tier,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,...,p1_summonerId,p2_summonerId,p3_summonerId,p4_summonerId,p5_summonerId,p6_summonerId,p7_summonerId,p8_summonerId,p9_summonerId,p10_summonerId
39767,MASTER,3310507770,NA1,1.582961e+12,1847.0,420.0,11.0,13.0,10.4.308.9400,CLASSIC,...,7D6lZrl-uPoRs4mnWMTFdFT1U_KkDbxvPwljWp8vJ9fHOCr-,Rt6AIvrlOOT83S8vCW0HlkhDKI530UFAIlvRXwy1oZzQUvI,PL9F8Wir3p0GFIbS0fXlTLu1XfPfPbC6w3-p8KZBk9jYsTsP,riZReuqHA4MdPH7t9DDZjagdoXB5d4dhvWxawQrxNfZSIeI,WwPzERUXDYWsRTonndghd6wA2kI-9-I1ymSuMltlliMFqpg,iz80TA6UuzOH93qOhcFOKs8dIREqXa196vMniHX0Kp1lGQE,hpQ3suiurulv5b9VE9SjAmAfBDfiWymW8LllfoftCS-KEIM,O1F9xsbExZJhhXHFhqFDceALQE7FtDvElPLFuGS093s_Vow,IgY_jy8fXWrTqq__MnXxySyUFHykdKaFP_FCwbeBVMPZUspj,5a8noOVBzjEneWB3LWwYlBxPZuyJI1QN-yXY-TeMfssj1Vc


In [46]:
dfDupe.append(d)

,tier,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,...,p1_summonerId,p2_summonerId,p3_summonerId,p4_summonerId,p5_summonerId,p6_summonerId,p7_summonerId,p8_summonerId,p9_summonerId,p10_summonerId
39767,MASTER,3310507770,NA1,1.582961e+12,1847.0,420.0,11.0,13.0,10.4.308.9400,CLASSIC,...,7D6lZrl-uPoRs4mnWMTFdFT1U_KkDbxvPwljWp8vJ9fHOCr-,Rt6AIvrlOOT83S8vCW0HlkhDKI530UFAIlvRXwy1oZzQUvI,PL9F8Wir3p0GFIbS0fXlTLu1XfPfPbC6w3-p8KZBk9jYsTsP,riZReuqHA4MdPH7t9DDZjagdoXB5d4dhvWxawQrxNfZSIeI,WwPzERUXDYWsRTonndghd6wA2kI-9-I1ymSuMltlliMFqpg,iz80TA6UuzOH93qOhcFOKs8dIREqXa196vMniHX0Kp1lGQE,hpQ3suiurulv5b9VE9SjAmAfBDfiWymW8LllfoftCS-KEIM,O1F9xsbExZJhhXHFhqFDceALQE7FtDvElPLFuGS093s_Vow,IgY_jy8fXWrTqq__MnXxySyUFHykdKaFP_FCwbeBVMPZUspj,5a8noOVBzjEneWB3LWwYlBxPZuyJI1QN-yXY-TeMfssj1Vc


In [57]:
temp = df

In [ ]:
status = df['gameId'].duplicated()
dupes = status[status==True]
while len(dupes)>0:
    
    
    
    status = df['gameId'].duplicated()
    dupes = status[status==True]